## Chapter 4 

### 4.1 User Feature Engineering

One of the reasons why I decided to use the Ponpare dataset is because is allows to dive properly into feature engineering tasks. The code here will be an illustration. Let's start as always defining input and output paths.

In [1]:
import pandas as pd
import numpy as np
import pickle
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()
%matplotlib inline

from functools import reduce
from scipy import stats
from collections import Counter

inp_dir ="../datasets/Ponpare/data_processed"
out_dir = "../datasets/Ponpare/data_processed"

As mentioned in previous chapters, here we will consider **ONLY** users that have been active during training. Of course there is a caveat here that users that registered a long time ago had more time to interact. Therefore, we might be ignoring in this analysis good users that registered recently but had no time to interact. As I mentioned in Chapter 1, in the real world one has to recommend to everyone. There I also suggested possible solutions to this issue. For the time being and in moving forward, let's focus on *"active"* users defined as those that have at least interacted once (whether visiting or buying) during training.

In [2]:
# Interactions
df_visits_train = pd.read_pickle(os.path.join(inp_dir, 'train', 'df_visits_train.p'))
df_purchases_train = pd.read_pickle(os.path.join(inp_dir, 'train', 'df_purchases_train.p'))

# User features
df_users_train = pd.read_pickle(os.path.join(inp_dir, 'train', 'df_users_train.p'))

# train coupons features
df_coupon_train = pd.read_pickle(os.path.join(inp_dir, 'train', 'df_coupons_train_feat.p'))

# dictionaty of mappings
dict_of_mappings = pickle.load(open(os.path.join(inp_dir, 'dict_of_mappings.p'), 'rb') )

# let's see if there are users in training that did nothing. No visits, no purchases
active_users = list(
    set(
        list(df_visits_train.user_id_hash.unique()) +
        list(df_purchases_train.user_id_hash.unique())
        )
    )
inactive_users = np.setdiff1d(list(df_users_train.user_id_hash.unique()), active_users)

print(inactive_users.shape)
print(len(df_users_train))

(66,)
22690


Luckily, we only ignore 66 users/customers out of 22690. 

Below we see that there are 922 users that have withdrawn. However, we still keep them since we still want to learn from their behaviour.

In [3]:
# df_users_train_active (df_utr)
df_utr = df_users_train[~df_users_train.user_id_hash.isin(inactive_users)]
withdraw_users = df_utr[~df_utr.withdraw_date.isna()]['user_id_hash'].unique()

# there are 922 users that have withdrawn, but we still keep them!
# We still want to learn from their behaviour

# drop unneccesary columns
df_utr.drop(['reg_date','withdraw_date','days_to_present'], axis=1, inplace=True)

# df_visits_train_active (df_vtr)
df_vtr = df_visits_train[df_visits_train.user_id_hash.isin(df_utr.user_id_hash)]

# df_purchases_train_active (df_ptr)
df_ptr = df_purchases_train[df_purchases_train.user_id_hash.isin(df_utr.user_id_hash)]

/usr/local/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


There are 2117 coupons in the purchased table that are not in the visits table. Maybe this is due to the fact that these coupons where purchased without previous visits. A way to check this would be to see if all purchased coupons in the visits table appear more than once (i.e. if the minimum number of visits for those coupons is 2). Let's do it

In [4]:
visits_purchased = df_vtr[~df_vtr.purchaseid_hash.isna()]['view_coupon_id_hash'].unique()
purchased_not_in_visits = df_ptr[~df_ptr.coupon_id_hash.isin(visits_purchased)]['coupon_id_hash'].unique()
purchased_min_visits = (df_vtr[df_vtr.view_coupon_id_hash.isin(visits_purchased)]['view_coupon_id_hash']
    .value_counts()
    .min())

print(purchased_min_visits)

2


Let's start with the proper feature engineering

### Demographics-based features 

These with be **denoted with "_d"** (e.g. df_user_feat_d)  

In [5]:
# User features based on "demographics" (_d)
df_user_feat_d = df_utr.copy()

df_user_feat_d['pref_name_cat'] = df_user_feat_d.pref_name.replace(dict_of_mappings['ken_name_cat'])

# new category for NaN (again, you might prefer another method to fill NaN)
new_pref_name_cat = df_user_feat_d.pref_name_cat.max() + 1
df_user_feat_d['pref_name_cat'] = df_user_feat_d.pref_name_cat.fillna(new_pref_name_cat).astype('int')
df_user_feat_d.drop('pref_name', axis=1, inplace=True)

# given that we have added a category to pref_name, let's update the dict_of_mappings accordingly
pref_name_dict = dict_of_mappings['ken_name_cat'].copy()
pref_name_dict['NAN'] = int(new_pref_name_cat)
dict_of_mappings['pref_name_cat'] = pref_name_dict

dict_of_mappings['sex_id_cat'] = {'f':0, 'm':1}
df_user_feat_d['sex_id_cat'] = df_user_feat_d.sex_id.replace(dict_of_mappings['sex_id_cat'])
df_user_feat_d.drop('sex_id', axis=1, inplace=True)

df_user_feat_d.head()

,age,user_id_hash,pref_name_cat,sex_id_cat
0,25,d9dca3cb44bab12ba313eaa681f663eb,47,0
1,34,560574a339f1b25e57b0221e486907ed,2,0
2,41,e66ae91b978b3229f8fd858c80615b73,4,1
3,25,43fc18f32eafb05713ec02935e2c2825,47,1
4,62,dc6df8aa860f8db0d710ce9d4839840f,5,1


Let's have a look. You can do the same throughout all the process for other features.

In [16]:
print(dict_of_mappings['pref_name_cat'])

{'saitama': 0, 'chiba': 1, 'tokyo': 2, 'kyoto': 3, 'aichi': 4, 'kanagawa': 5, 'hokkaido': 6, 'hukuoka': 7, 'tochigi': 8, 'osaka': 9, 'miyagi': 10, 'hukushima': 11, 'ooita': 12, 'kouchi': 13, 'hiroshima': 14, 'niigata': 15, 'okayama': 16, 'ehime': 17, 'kagawa': 18, 'tokushima': 19, 'hyogo': 20, 'gihu': 21, 'miyazaki': 22, 'nagasaki': 23, 'ishikawa': 24, 'yamagata': 25, 'shizuoka': 26, 'aomori': 27, 'okinawa': 28, 'akita': 29, 'nagano': 30, 'iwate': 31, 'kumamoto': 32, 'yamaguchi': 33, 'saga': 34, 'nara': 35, 'mie': 36, 'gunma': 37, 'wakayama': 38, 'yamanashi': 39, 'tottori': 40, 'kagoshima': 41, 'hukui': 42, 'shiga': 43, 'toyama': 44, 'shimane': 45, 'ibaraki': 46, 'NAN': 47}


In [12]:
pref_name_cat_inv_idx = {v:k for k,v in dict_of_mappings['pref_name_cat'].items()}

top20_pref_name_idx = list(df_user_feat_d.pref_name_cat.value_counts()[:20].index)
top20_pref_name = [pref_name_cat_inv_idx[i] for i in top20_pref_name_idx]
top20_pref_name_cnts = list(df_user_feat_d.pref_name_cat.value_counts()[:20].values)

pd.DataFrame({'pref_name': top20_pref_name, 'Counts':top20_pref_name_cnts})


,pref_name,Counts
0,NAN,7168
1,tokyo,2805
2,kanagawa,1628
3,osaka,1626
4,aichi,931
5,hyogo,872
6,saitama,869
7,chiba,824
8,hukuoka,720
9,hokkaido,619


### Purchase behaviour-based features 

These will be **denoted with _p**

In [23]:
# Let's first define a couple of functions that will be useful later
def top_values(row, top_n=2):
    """
    Helper that returns the top_n values for a given row when the row is a list of items
    """
    counts = [c[0] for c in Counter(row).most_common()]
    row_len = len(set(row))
    if row_len < top_n:
        top_vals = counts + [counts[-1]]*(top_n - row_len)
    else:
        top_vals = counts[:top_n]
    return top_vals


def time_diff(row, all_metrics=False):
    """
    Helper that returns the time difference between the elements of a given row when the row is a list of dates
    """
    if len(row) == 1:
        min_diff = 0
        max_diff = 0
        median_diff = 0
    else:
        row = sorted(row, reverse=True)
        diff = [t - s for t, s in zip(row, row[1:])]
        min_diff = np.min(diff)
        max_diff = np.max(diff)
        median_diff = np.median(diff)
    if all_metrics:
        return [min_diff, max_diff, median_diff]
    else:
        return median_diff

In [24]:
df_ptr['day_of_week'] = df_ptr.i_date.dt.dayofweek

# a collection of aggregation functions
agg_functions_p = {
    'purchaseid_hash': ['count'],
    'coupon_id_hash': ['nunique'],
    'item_count': ['sum'],
    'small_area_name': ['nunique'],
    'day_of_week': ['nunique']
    }
df_user_feat_p = df_ptr.groupby("user_id_hash").agg(agg_functions_p)

# readable column names
df_user_feat_p.columns =  ["_".join(pair) for pair in df_user_feat_p.columns]
df_user_feat_p.reset_index(inplace=True)

In [25]:
# median time difference between purchases
time_diff_df_p = (df_ptr.groupby("user_id_hash")['days_to_present']
    .apply(list)
    .reset_index()
    )
time_diff_df_p['median_time_diff'] = time_diff_df_p.days_to_present.apply(lambda x: time_diff(x))
time_diff_df_p.drop('days_to_present', axis=1, inplace=True)

In [26]:
# top 2 small_area name of shop location and days of week for purchases
small_area_df_p = (df_ptr.groupby("user_id_hash")['small_area_name']
    .apply(list)
    .reset_index()
    )
small_area_df_p['top_small_areas'] = small_area_df_p.small_area_name.apply(lambda x: top_values(x))
small_area_df_p['top1_small_area_name'] =  small_area_df_p.top_small_areas.apply(lambda x: x[0])
small_area_df_p['top2_small_area_name'] =  small_area_df_p.top_small_areas.apply(lambda x: x[1])
small_area_df_p.drop(['small_area_name', 'top_small_areas'], axis=1, inplace=True)

# replace with corresponding categories
for col in ['top1_small_area_name', 'top2_small_area_name']:
    small_area_df_p[col] = small_area_df_p[col].replace(dict_of_mappings['small_area_name_cat'])

In [27]:
# Day of the week related features
day_of_week_df_p = (df_ptr.groupby("user_id_hash")['day_of_week']
    .apply(list)
    .reset_index()
    )
day_of_week_df_p['top_days_of_week'] = day_of_week_df_p.day_of_week.apply(lambda x: top_values(x))
day_of_week_df_p['top1_dayofweek'] =  day_of_week_df_p.top_days_of_week.apply(lambda x: x[0])
day_of_week_df_p['top2_dayofweek'] =  day_of_week_df_p.top_days_of_week.apply(lambda x: x[1])
day_of_week_df_p.drop(['day_of_week', 'top_days_of_week'], axis=1, inplace=True)

In [28]:
# merge all together
df_l_p = [df_user_feat_p, time_diff_df_p, small_area_df_p, day_of_week_df_p]
df_user_feat_p = reduce(lambda left,right: pd.merge(left,right,on=['user_id_hash']), df_l_p)

# add "_cat" to categorical features
cat_feat_p = ['top1_small_area_name', 'top2_small_area_name', 'top1_dayofweek', 'top2_dayofweek']
cat_feat_name_p = [c+"_cat" for c in cat_feat_p]
cat_feat_name_dict_p = dict(zip(cat_feat_p, cat_feat_name_p))
df_user_feat_p.rename(index=str, columns=cat_feat_name_dict_p, inplace=True)

df_user_feat_p.head()

,user_id_hash,purchaseid_hash_count,coupon_id_hash_nunique,item_count_sum,small_area_name_nunique,day_of_week_nunique,median_time_diff,top1_small_area_name_cat,top2_small_area_name_cat,top1_dayofweek_cat,top2_dayofweek_cat
0,0000b53e182165208887ba65c079fc21,1,1,2,1,1,0.0,11,11,4,4
1,00035b86e6884589ec8d28fbf2fe7757,1,1,2,1,1,0.0,8,8,3,3
2,0005b1068d5f2b8f2a7c978fcfe1ca06,2,2,2,2,1,0.0,2,24,3,3
3,000cc06982785a19e2a2fdb40b1c9d59,21,21,25,9,6,6.0,26,51,2,4
4,0013518e41c416cd6a181d277dd8ca0b,4,4,4,2,3,10.0,2,15,6,2


### Visit behaviour-like features 

These will be **denoted with _v**

In [29]:
# User features based on visit behaviour (_v):
df_vtr['day_of_week'] = df_vtr.i_date.dt.dayofweek

agg_functions_v = {
    'view_coupon_id_hash': ['count', 'nunique'],
    'session_id_hash': ['nunique'],
    'day_of_week': ['nunique']
    }
df_user_feat_v = df_vtr.groupby("user_id_hash").agg(agg_functions_v)
df_user_feat_v.columns =  ["_".join(pair) for pair in df_user_feat_v.columns]
df_user_feat_v.reset_index(inplace=True)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [30]:
# min/max/median time difference between visits
time_diff_df_v = (df_vtr.groupby("user_id_hash")['days_to_present']
    .apply(list)
    .reset_index()
    )
time_diff_df_v['time_diff'] = time_diff_df_v.days_to_present.apply(lambda x: time_diff(x, all_metrics=True))
time_diff_df_v['min_time_diff'] = time_diff_df_v.time_diff.apply(lambda x: x[0])
time_diff_df_v['max_time_diff'] = time_diff_df_v.time_diff.apply(lambda x: x[1])
time_diff_df_v['median_time_diff'] = time_diff_df_v.time_diff.apply(lambda x: x[2])
time_diff_df_v.drop(['days_to_present','time_diff'], axis=1, inplace=True)

In [31]:
# top 2 days of week for visits
day_of_week_df_v = (df_vtr.groupby("user_id_hash")['day_of_week']
    .apply(list)
    .reset_index()
    )
day_of_week_df_v['top_days_of_week'] = day_of_week_df_v.day_of_week.apply(lambda x: top_values(x))
day_of_week_df_v['top1_dayofweek'] =  day_of_week_df_v.top_days_of_week.apply(lambda x: x[0])
day_of_week_df_v['top2_dayofweek'] =  day_of_week_df_v.top_days_of_week.apply(lambda x: x[1])
day_of_week_df_v.drop(['day_of_week', 'top_days_of_week'], axis=1, inplace=True)

In [32]:
# merge all together
df_l_v = [df_user_feat_v, time_diff_df_v, day_of_week_df_v]
df_user_feat_v = reduce(lambda left,right: pd.merge(left,right,on=['user_id_hash']), df_l_v)

cat_feat_v = ['top1_dayofweek', 'top2_dayofweek']
cat_feat_name_v = [c+"_cat" for c in cat_feat_v]
cat_feat_name_dict_v = dict(zip(cat_feat_v, cat_feat_name_v))
df_user_feat_v.rename(index=str, columns=cat_feat_name_dict_v, inplace=True)

# in the case of visits, let's add view to the beggining of all columns
visits_cols = df_user_feat_v.columns.tolist()
visits_cols = visits_cols[:3] + ['view_'+c for c in visits_cols[3:]]
df_user_feat_v.columns = visits_cols

df_user_feat_v.head()

,user_id_hash,view_coupon_id_hash_count,view_coupon_id_hash_nunique,view_session_id_hash_nunique,view_day_of_week_nunique,view_min_time_diff,view_max_time_diff,view_median_time_diff,view_top1_dayofweek_cat,view_top2_dayofweek_cat
0,0000b53e182165208887ba65c079fc21,75,55,11,6,0,43,0.0,5,4
1,00035b86e6884589ec8d28fbf2fe7757,2,1,1,1,0,0,0.0,3,3
2,0005b1068d5f2b8f2a7c978fcfe1ca06,6,2,2,2,0,2,0.0,3,1
3,000cc06982785a19e2a2fdb40b1c9d59,267,157,137,7,0,14,0.0,4,3
4,0013518e41c416cd6a181d277dd8ca0b,29,14,15,6,0,86,0.0,3,6


### Coupon-based user features 

These will be **denoted by _c**

Here possibilities are nearly endless. Let's focus on price features (`catalog_price` and `price_rate`) and coupon category features: `capsule_text_cat` and `genre_name_cat`. We will remove purchases from the visits data because we will build features separately for purchased and visits.

The following piece of code is a bit "heavy" (maybe not). In reality there is nothing complex in there. Let's go!

In [33]:
df_vtr_visits = df_vtr.copy()

# let's create an id to easily drop purchases from the visits data because we will 
# build features separately for purchased and visits
df_vtr_visits['activity_hash'] = df_vtr_visits['user_id_hash'] + "_" + df_vtr_visits['view_coupon_id_hash']
purchases = df_vtr_visits[~df_vtr_visits.purchaseid_hash.isna()]['activity_hash'].unique()
df_vtr_visits = df_vtr_visits[~df_vtr_visits.activity_hash.isin(purchases)][['user_id_hash','view_coupon_id_hash']]
df_vtr_visits.columns = ['user_id_hash','coupon_id_hash']

df_coupon_based_feat_l = []
for name,df in [('purchase', df_ptr), ('visits', df_vtr_visits)]:

    list_user = ['user_id_hash', 'coupon_id_hash']
    list_coupons = ['coupon_id_hash', 'catalog_price', 'discount_price', 'catalog_price_cat',
        'discount_price_cat', 'capsule_text_cat', 'genre_name_cat']

    # merge correponding interaction df with coupon features. Then aggregate
    df_c = pd.merge(df[list_user], df_coupon_train[list_coupons], on='coupon_id_hash')
    agg_functions_p_c = {
        'catalog_price': ['mean', 'median', 'min', 'max'],
        'discount_price': ['mean', 'median', 'min', 'max'],
        }
    df_price_num = df_c.groupby('user_id_hash').agg(agg_functions_p_c)
    df_price_num.columns =  ["_".join(pair) for pair in df_price_num.columns]
    df_price_num.reset_index(inplace=True)

    # let's find out the number of unique price and discount categories that customers interacted with
    tmp_df_l_1 = []
    for col in ['catalog_price_cat', 'discount_price_cat']:
        tmp_df = df_c.pivot_table(values='coupon_id_hash', index='user_id_hash',
            columns=col, aggfunc= lambda x: len(x.unique()))
        colname = col.split("_cat")[0]
        colnames = ["_".join([colname,str(cat)]) for cat in tmp_df.columns.categories]
        tmp_df.columns = colnames
        tmp_df.reset_index(inplace=True)
        tmp_df.fillna(0, inplace=True)
        tmp_df_l_1.append(tmp_df)
    df_price_cat = reduce(lambda left,right: pd.merge(left,right,on=['user_id_hash']), tmp_df_l_1)

    # now let's find out the top N types of coupons that customers interacted with by capsule and genre  
    tmp_df_l_2 = []
    top_n = 3
    for col in ['capsule_text_cat', 'genre_name_cat']:
        tmp_df = df_c.groupby('user_id_hash')[col].apply(list).reset_index()
        root = col.split("_cat")[0]
        colname = "top_" + root
        colnames = ["top"+str(i)+"_"+root for i in range(1,top_n+1)]
        tmp_df[colname] = tmp_df[col].apply(lambda x: top_values(x, top_n=top_n))
        for i,cn in enumerate(colnames):
            tmp_df[cn] = tmp_df[colname].apply(lambda x: x[i])
        tmp_df.drop([col, colname], axis=1, inplace=True)
        tmp_df_l_2.append(tmp_df)
    df_type_cat = reduce(lambda left,right: pd.merge(left,right,on=['user_id_hash']), tmp_df_l_2)
    tmp_colnames = df_type_cat.columns.tolist()
    cat_colnames = tmp_colnames[:1]+[c+"_cat" for c in tmp_colnames[1:]]
    df_type_cat.columns = cat_colnames

    # append the numeric, price category and type category related features
    df_l_c = [df_price_num, df_price_cat, df_type_cat]
    fdf = reduce(lambda left,right: pd.merge(left,right,on=['user_id_hash']), df_l_c)
    old_colnames = fdf.columns.tolist()
    
    if name is 'visits':
        new_colnames = old_colnames[:1] + ['view_'+c for c in old_colnames[1:]]
        fdf.columns = new_colnames

    df_coupon_based_feat_l.append(fdf)

In [34]:
# lets have a look to what is there
df_coupon_based_feat_l[0].head()

,user_id_hash,catalog_price_mean,catalog_price_median,catalog_price_min,catalog_price_max,discount_price_mean,discount_price_median,discount_price_min,discount_price_max,catalog_price_0,catalog_price_1,catalog_price_2,discount_price_0,discount_price_1,discount_price_2,top1_capsule_text_cat,top2_capsule_text_cat,top3_capsule_text_cat,top1_genre_name_cat,top2_genre_name_cat,top3_genre_name_cat
0,0000b53e182165208887ba65c079fc21,7900.000000,7900.0,7900,7900,2980.000000,2980.0,2980,2980,1.0,0.0,0.0,0.0,0.0,1.0,0,0,0,0,0,0
1,00035b86e6884589ec8d28fbf2fe7757,3110.000000,3110.0,3110,3110,1490.000000,1490.0,1490,1490,0.0,1.0,0.0,1.0,0.0,0.0,0,0,0,0,0,0
2,0005b1068d5f2b8f2a7c978fcfe1ca06,36780.000000,36780.0,1060,72500,9950.000000,9950.0,100,19800,0.0,1.0,1.0,1.0,1.0,0.0,8,7,7,8,7,7
3,000cc06982785a19e2a2fdb40b1c9d59,3997.666667,2980.0,1,14700,1397.380952,1190.0,0,4410,3.0,16.0,2.0,13.0,2.0,6.0,6,8,9,6,8,9
4,0013518e41c416cd6a181d277dd8ca0b,12623.750000,1975.0,500,46045,4350.000000,650.0,100,16000,0.0,3.0,1.0,3.0,1.0,0.0,9,10,11,9,10,11


In [35]:
df_coupon_based_feat_l[1].head()

,user_id_hash,view_catalog_price_mean,view_catalog_price_median,view_catalog_price_min,view_catalog_price_max,view_discount_price_mean,view_discount_price_median,view_discount_price_min,view_discount_price_max,view_catalog_price_0,view_catalog_price_1,view_catalog_price_2,view_discount_price_0,view_discount_price_1,view_discount_price_2,view_top1_capsule_text_cat,view_top2_capsule_text_cat,view_top3_capsule_text_cat,view_top1_genre_name_cat,view_top2_genre_name_cat,view_top3_genre_name_cat
0,0000b53e182165208887ba65c079fc21,11185.925373,6000.0,1773,49800,4100.641791,2945.0,525,15000,14.0,15.0,20.0,14.0,18.0,17.0,0,6,11,0,11,6
1,0005b1068d5f2b8f2a7c978fcfe1ca06,500.000000,500.0,500,500,100.000000,100.0,100,100,1.0,0.0,0.0,1.0,0.0,0.0,8,8,8,8,8,8
2,000cc06982785a19e2a2fdb40b1c9d59,5849.414474,4000.0,500,60900,2593.026316,1910.0,100,29800,80.0,28.0,17.0,75.0,35.0,15.0,0,6,8,0,6,8
3,0013518e41c416cd6a181d277dd8ca0b,10105.454545,11000.0,2000,20000,4512.727273,4500.0,100,9990,2.0,2.0,5.0,1.0,2.0,6.0,11,0,10,11,0,10
4,001acdee812a18acfd7509172bed5700,10175.266667,5040.0,1000,80500,3271.680000,2100.0,500,19800,28.0,22.0,21.0,33.0,22.0,16.0,6,0,1,6,0,11


And you guessed right, let's merge the two

In [36]:
df_user_feat_c = reduce(lambda left,right: pd.merge(left,right,on=['user_id_hash'],how='outer'), 
                        df_coupon_based_feat_l)

### Putting all together

So far we have features based on demographic information, purchase behaviour, visit behaviour and also features that are only based on the type of coupons that they interacted with (whether purchased or visited). Again, you guessed right, let's merge all the datasets into a "big", wide dataset. 

In [37]:
final_list_of_dfs = [df_user_feat_d, df_user_feat_c, df_user_feat_v, df_user_feat_p]
df_user_feat = reduce(lambda left,right: pd.merge(left,right,on=['user_id_hash'],how='outer'), final_list_of_dfs)

df_user_feat.shape

(22624, 63)

In [38]:
df_user_feat.head()

,age,user_id_hash,pref_name_cat,sex_id_cat,catalog_price_mean,catalog_price_median,catalog_price_min,catalog_price_max,discount_price_mean,discount_price_median,discount_price_min,discount_price_max,catalog_price_0,catalog_price_1,catalog_price_2,discount_price_0,discount_price_1,discount_price_2,top1_capsule_text_cat,top2_capsule_text_cat,top3_capsule_text_cat,top1_genre_name_cat,top2_genre_name_cat,top3_genre_name_cat,view_catalog_price_mean,view_catalog_price_median,view_catalog_price_min,view_catalog_price_max,view_discount_price_mean,view_discount_price_median,view_discount_price_min,view_discount_price_max,view_catalog_price_0,view_catalog_price_1,view_catalog_price_2,view_discount_price_0,view_discount_price_1,view_discount_price_2,view_top1_capsule_text_cat,view_top2_capsule_text_cat,view_top3_capsule_text_cat,view_top1_genre_name_cat,view_top2_genre_name_cat,view_top3_genre_name_cat,view_coupon_id_hash_count,view_coupon_id_hash_nunique,view_session_id_hash_nunique,view_day_of_week_nunique,view_min_time_diff,view_max_time_diff,view_median_time_diff,view_top1_dayofweek_cat,view_top2_dayofweek_cat,purchaseid_hash_count,coupon_id_hash_nunique,item_count_sum,small_area_name_nunique,day_of_week_nunique,median_time_diff,top1_small_area_name_cat,top2_small_area_name_cat,top1_dayofweek_cat,top2_dayofweek_cat
0,25,d9dca3cb44bab12ba313eaa681f663eb,47,0,7200.000000,7200.0,7200.0,7200.0,1575.000000,1575.0,1575.0,1575.0,1.0,0.0,0.0,1.0,0.0,0.0,6.0,6.0,6.0,6.0,6.0,6.0,26280.000000,10200.0,4480.0,80000.0,3044.285714,1990.0,1480.0,4980.0,1.0,1.0,2.0,1.0,2.0,1.0,6.0,2.0,2.0,6.0,2.0,2.0,13.0,6.0,1.0,1.0,0.0,0.0,0.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,28.0,28.0,2.0,2.0
1,34,560574a339f1b25e57b0221e486907ed,2,0,3619.133333,3000.0,320.0,12000.0,1308.666667,1000.0,160.0,4500.0,2.0,7.0,2.0,9.0,1.0,1.0,0.0,8.0,6.0,0.0,8.0,6.0,7375.390625,4400.0,920.0,44625.0,2874.140625,1997.5,315.0,9900.0,31.0,12.0,11.0,26.0,17.0,11.0,0.0,10.0,9.0,0.0,10.0,11.0,120.0,70.0,59.0,7.0,0.0,37.0,0.0,0.0,1.0,15.0,11.0,19.0,4.0,7.0,15.5,4.0,12.0,5.0,3.0
2,41,e66ae91b978b3229f8fd858c80615b73,4,1,2535.000000,2535.0,2535.0,2535.0,1500.000000,1500.0,1500.0,1500.0,0.0,1.0,0.0,1.0,0.0,0.0,6.0,6.0,6.0,6.0,6.0,6.0,2600.000000,2600.0,2600.0,2600.0,1200.000000,1200.0,1200.0,1200.0,1.0,0.0,0.0,1.0,0.0,0.0,6.0,6.0,6.0,6.0,6.0,6.0,8.0,2.0,4.0,2.0,0.0,7.0,0.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,6.0,6.0,4.0,4.0
3,25,43fc18f32eafb05713ec02935e2c2825,47,1,500.750000,1.0,1.0,2000.0,250.000000,0.0,0.0,1000.0,0.0,4.0,0.0,4.0,0.0,0.0,9.0,9.0,9.0,9.0,9.0,9.0,2398.333333,1560.0,1330.0,4305.0,916.000000,648.0,500.0,1600.0,3.0,0.0,0.0,3.0,0.0,0.0,6.0,9.0,0.0,6.0,9.0,0.0,6.0,6.0,4.0,3.0,0.0,95.0,8.0,3.0,0.0,4.0,4.0,4.0,2.0,3.0,48.0,2.0,33.0,0.0,2.0
4,62,dc6df8aa860f8db0d710ce9d4839840f,5,1,8426.909091,4500.0,750.0,37800.0,4198.181818,2250.0,350.0,18900.0,4.0,7.0,6.0,5.0,7.0,5.0,0.0,6.0,12.0,0.0,11.0,6.0,8853.020243,5500.0,320.0,61600.0,4129.080972,2500.0,100.0,30800.0,187.0,91.0,106.0,183.0,75.0,126.0,0.0,12.0,6.0,11.0,0.0,6.0,785.0,478.0,274.0,7.0,0.0,13.0,0.0,1.0,4.0,22.0,17.0,36.0,9.0,7.0,5.0,7.0,15.0,1.0,3.0


Before we wrap up this chapter, there is something else we need to consider. There are 116 users in training that visit but never bought

In [39]:
df_utr[(df_utr.user_id_hash.isin(df_vtr.user_id_hash)) & \
       (~df_utr.user_id_hash.isin(df_ptr.user_id_hash))].shape[0]

116

For them *all* columns in the `df_user_feat_p` will be NaN. I am going to fill them with -1. 

In [40]:
prefixes = ['top1_', 'top2_', 'top3_']
for col in df_user_feat.columns[2:]:
    if col.endswith("_cat") and df_user_feat[col].isna().any():
        # Since we are going to treat NaN as a new category for these cols
        # we need to update the dictionary of mappings. They are mostly
        # related to capsule_text_cat and genre_name_cat with the prefix
        # top_n:
        for prefix in prefixes:
            if prefix in col:
                start = re.search(prefix, col).end()
        root_name = col[start:]

        # if the column is derived from root_name, the correponding
        # dictionary would be the same as that of root_name plus an extra
        # category for 'NAN'
        if root_name in dict_of_mappings.keys():
            col_dict = dict_of_mappings[root_name].copy()
            new_col_cat = len(col_dict)
        else:
            col_categories = np.sort(df_user_feat[col].unique())[:-1]
            col_dict = {int(k):int(v) for v,k in enumerate(col_categories)}
            new_col_cat =int(df_user_feat[col].max()+1)

        col_dict['NAN'] = new_col_cat
        dict_of_mappings[col] = col_dict
        df_user_feat[col] = df_user_feat[col].fillna(new_col_cat).astype('int')

    else:
        df_user_feat[col].fillna(-1, inplace=True)

In [41]:
df_user_feat.head()

,age,user_id_hash,pref_name_cat,sex_id_cat,catalog_price_mean,catalog_price_median,catalog_price_min,catalog_price_max,discount_price_mean,discount_price_median,discount_price_min,discount_price_max,catalog_price_0,catalog_price_1,catalog_price_2,discount_price_0,discount_price_1,discount_price_2,top1_capsule_text_cat,top2_capsule_text_cat,top3_capsule_text_cat,top1_genre_name_cat,top2_genre_name_cat,top3_genre_name_cat,view_catalog_price_mean,view_catalog_price_median,view_catalog_price_min,view_catalog_price_max,view_discount_price_mean,view_discount_price_median,view_discount_price_min,view_discount_price_max,view_catalog_price_0,view_catalog_price_1,view_catalog_price_2,view_discount_price_0,view_discount_price_1,view_discount_price_2,view_top1_capsule_text_cat,view_top2_capsule_text_cat,view_top3_capsule_text_cat,view_top1_genre_name_cat,view_top2_genre_name_cat,view_top3_genre_name_cat,view_coupon_id_hash_count,view_coupon_id_hash_nunique,view_session_id_hash_nunique,view_day_of_week_nunique,view_min_time_diff,view_max_time_diff,view_median_time_diff,view_top1_dayofweek_cat,view_top2_dayofweek_cat,purchaseid_hash_count,coupon_id_hash_nunique,item_count_sum,small_area_name_nunique,day_of_week_nunique,median_time_diff,top1_small_area_name_cat,top2_small_area_name_cat,top1_dayofweek_cat,top2_dayofweek_cat
0,25,d9dca3cb44bab12ba313eaa681f663eb,47,0,7200.000000,7200.0,7200.0,7200.0,1575.000000,1575.0,1575.0,1575.0,1.0,0.0,0.0,1.0,0.0,0.0,6,6,6,6,6,6,26280.000000,10200.0,4480.0,80000.0,3044.285714,1990.0,1480.0,4980.0,1.0,1.0,2.0,1.0,2.0,1.0,6,2,2,6,2,2,13.0,6.0,1.0,1.0,0.0,0.0,0.0,2,2,1.0,1.0,1.0,1.0,1.0,0.0,28,28,2,2
1,34,560574a339f1b25e57b0221e486907ed,2,0,3619.133333,3000.0,320.0,12000.0,1308.666667,1000.0,160.0,4500.0,2.0,7.0,2.0,9.0,1.0,1.0,0,8,6,0,8,6,7375.390625,4400.0,920.0,44625.0,2874.140625,1997.5,315.0,9900.0,31.0,12.0,11.0,26.0,17.0,11.0,0,10,9,0,10,11,120.0,70.0,59.0,7.0,0.0,37.0,0.0,0,1,15.0,11.0,19.0,4.0,7.0,15.5,4,12,5,3
2,41,e66ae91b978b3229f8fd858c80615b73,4,1,2535.000000,2535.0,2535.0,2535.0,1500.000000,1500.0,1500.0,1500.0,0.0,1.0,0.0,1.0,0.0,0.0,6,6,6,6,6,6,2600.000000,2600.0,2600.0,2600.0,1200.000000,1200.0,1200.0,1200.0,1.0,0.0,0.0,1.0,0.0,0.0,6,6,6,6,6,6,8.0,2.0,4.0,2.0,0.0,7.0,0.0,4,1,1.0,1.0,1.0,1.0,1.0,0.0,6,6,4,4
3,25,43fc18f32eafb05713ec02935e2c2825,47,1,500.750000,1.0,1.0,2000.0,250.000000,0.0,0.0,1000.0,0.0,4.0,0.0,4.0,0.0,0.0,9,9,9,9,9,9,2398.333333,1560.0,1330.0,4305.0,916.000000,648.0,500.0,1600.0,3.0,0.0,0.0,3.0,0.0,0.0,6,9,0,6,9,0,6.0,6.0,4.0,3.0,0.0,95.0,8.0,3,0,4.0,4.0,4.0,2.0,3.0,48.0,2,33,0,2
4,62,dc6df8aa860f8db0d710ce9d4839840f,5,1,8426.909091,4500.0,750.0,37800.0,4198.181818,2250.0,350.0,18900.0,4.0,7.0,6.0,5.0,7.0,5.0,0,6,12,0,11,6,8853.020243,5500.0,320.0,61600.0,4129.080972,2500.0,100.0,30800.0,187.0,91.0,106.0,183.0,75.0,126.0,0,12,6,11,0,6,785.0,478.0,274.0,7.0,0.0,13.0,0.0,1,4,22.0,17.0,36.0,9.0,7.0,5.0,7,15,1,3


In [42]:
# There are 18 users that are in active_users but not in df_user_feat:
np.setdiff1d(active_users, df_users_train.user_id_hash.unique()).size

# This is because these users are not in df_users_train

# note that the "train" is redundant here. Unlike coupons, there will not be
# user_feat_test in this excercise.
df_user_feat.to_pickle(os.path.join(out_dir,"train","df_users_train_feat.p"))

# save dictionary
pickle.dump(dict_of_mappings, open(os.path.join(out_dir, 'dict_of_mappings.p'), 'wb') )

Time to start recommending...